# 基本面选股

In [ ]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 机器学习模块
import xgboost as xgb

# 回测框架
import backtrader as bt
import backtrader.indicators as btind

# 基础函数
import utilsJ

## 基本面因子

### 盈利性
+ 毛利率
+ 净利率
+ ROE
+ 营收同比增长率
+ 净利润同比增长率
+ 扣非净利润

### 流动性
+ 资产负债率
+ 流动比率
+ 速动比率
+ 应收账款周转率
+ 应收账款账龄分布
+ 存货周转率

### 估值
+ PE倍数（市盈率）
+ PB倍数（市净率）
+ PEG

### 其他
+ 商誉占总资产比重
+ 商誉占净资产比重

# 全市场循环

In [ ]:
s_date = datetime.datetime.now()- datetime.timedelta(days=365*2)
e_date = datetime.datetime.now()

pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
stock_list = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code').ts_code

roe_list = {}
for s in stock_list:
    df = utilsJ_ML.stock_finance('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6',s,s_date, e_date)
    df.reset_index(inplace=True)
    try:
        df['roe_seasonal'] = df.roe_dt.diff()
        df.loc[[x.month == 3 for x in df.trade_date],'roe_seasonal'] = df[[x.month == 3 for x in df.trade_date]]['roe_dt']
        if df.roe_seasonal.iloc[-1] > df.roe_seasonal.iloc[-2] and df.roe_seasonal.iloc[-2] > df.roe_seasonal.iloc[-3] and (df.roe_seasonal.iloc[-1] + df.roe_seasonal.iloc[-2]) > (df.roe_seasonal.iloc[-3] + df.roe_seasonal.iloc[-4]):
            roe_list[s] = df.roe_dt.iloc[-1]
    except:
        print(s)
        print(df)
    time.sleep(0.8)
sorted(roe_list.items(), key=lambda x:x[1])[-int(np.round(len(roe_list) * 0.2)):-1]